In [1]:
from IPython.display import Image
import ipywidgets as widgets
import qgrid
import pandas as pd
import time
import warnings
from measurement_directory import *
from numpy import nan
from ipywidgets import interact
import os
from logparsing import translate_stringy_list

## Run the next two cells to load the log from one of today's runs.

In [2]:
# def translate_stringy_list(stringy_list):
#     #pd.read_csv imports listboundvariables dtype as a string, we want it in list form
#     final_list = []
#     stringy_list = stringy_list.split(',')
#     for var in stringy_list:
#         final_list.append(var.translate({ord(c): None for c in '[ ]\,\''}))
#     return final_list

def save_image_log(event, qgrid_widget):
    df_display = qgrid_widget.get_changed_df()
    qgrid_widget.close()
    df = pd.read_csv(filename)
    for idx in range(len(df_display)):
        for column in df_display.columns:
            df.loc[idx, column] = df_display.loc[idx, column]
    df.to_csv(filename, index = False)
    qgrid_widget = edit_image_log()
    qgrid_widget.on('cell_edited',save_image_log)
    display(qgrid_widget)

def edit_image_log(new_column = False, extra_display_vars = []):
    
#     try: #save the edited log if it was opened and edited
#         save_image_log(filename)
#     except:
#         pass
    df = pd.read_csv(filename, dtype={'badshot': 'bool'})
    display_vars = []
    
    if 'editable_variables' in df.columns:
        editable_vars = translate_stringy_list(df['editable_variables'][0])
    else:
        editable_vars = []
    
    if new_column:
        column_name = input('enter new column name (use CamelCase): ')
        if column_name not in df.columns:
            df[column_name] = nan
            editable_vars += [column_name]
            df['editable_variables'] = str(editable_vars)
        else:
            warnings.warn('column \'' + column_name + '\' already exists')
    
    list_bound_vars = translate_stringy_list(df['ListBoundVariables'][0])
    if 'editable_variables' in df.columns:
        editable_vars = translate_stringy_list(df['editable_variables'][0])
    else:
        editable_vars = []
        
    if 'analyzed_variables' in df.columns:
        analyzed_vars = translate_stringy_list(df['analyzed_variables'][0])
        #parse analyzed_vars
        for i in range(len(analyzed_vars)):
            if analyzed_vars[i] not in df.columns:
                for column in df.columns:
                    if analyzed_vars[i] == column.translate({ord(c): None for c in '[ ]\,\''}):
                        analyzed_vars[i] = column
    else:
        analyzed_vars = []
        
    display_vars += ['badshot', 'notes'] + editable_vars + list_bound_vars  + extra_display_vars + analyzed_vars + ['runtime']
    if 'editable_variables' in df.columns:
        display_vars += ['editable_variables']
    
    df_display = df[display_vars]
    col_opts = { 'editable': False }
    col_defs = {}
    for col_name in editable_vars + ['badshot', 'notes']:
        col_defs[col_name] = { 'editable': True }
    col_defs['badshot']['ColumnWidth'] = 50
    qgrid_widget = qgrid.show_grid(df_display, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100},
                                  column_options=col_opts, column_definitions=col_defs)
    return qgrid_widget

image_viewer = widgets.Output(layout={'border': '1px solid black'})
def show_selected_image(event, qgrid_widget):
    image_viewer.clear_output()
    row_idx = int(event['new'][0])
    df = pd.read_csv(filename)
    raw_image_filename = df.loc[row_idx, 'filename0']
    filepath = measurement_dir + '\\' + raw_image_filename.replace('.spe', '.jpg')
    with image_viewer:
        if os.path.exists(filepath) and '.jpg' in filepath:
            pil_img = Image(filepath)
            display(pil_img)
        else:
            display('no jpg preview at ' + filepath)


def add_new_column(new_column_in = False):
    global new_column
    new_column = new_column_in

mode_select_widget = interact(add_new_column,new_column_in=[False, True])

interactive(children=(Dropdown(description='new_column_in', options=(False, True), value=False), Output()), _d…

In [5]:
%matplotlib notebook
global filename
measurement_dir = measurement_directory()
filename = measurement_dir + r'\image_log.csv'
qgrid_widget = edit_image_log(new_column)
qgrid_widget.on('cell_edited',save_image_log)
qgrid_widget.on('selection_changed', show_selected_image)
display(qgrid_widget)
display(image_viewer)

Enter name for this set of runs:foo


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

Output(layout=Layout(border='1px solid black'))

In [24]:
display('no image')

'no image'

In [20]:
df

NameError: name 'df' is not defined